# Colab으로 2022-Mar 데이터 수집 후 진행

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# 네이버뉴스 1개 크롤링
import requests
import pandas as pd
from bs4 import BeautifulSoup
def news(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    media_element = soup.select_one('a.media_end_head_top_logo img')
    if media_element:
        media = media_element.get('title')
    else:
        media = None
    title_element = soup.select_one('h2#title_area')
    if title_element:
        title = title_element.text
    else:
        title = None
    date_element = soup.select_one('span.media_end_head_info_datestamp_time')
    if date_element:
        date = date_element.get('data-date-time')
    else:
        date = None
    content_element = soup.select_one('article#dic_area')
    if content_element:
        content = content_element.text.strip()
    else:
        content = None
    return {
        'title': title,
        'date': date,
        'media': media,
        'content': content,
        'url': url
    }
news('https://n.news.naver.com/mnews/article/001/0013674088?sid=104')
#네이버뉴스 페이지+원하는 날짜 크롤링
def news_list(keyword, startdate, enddate):
    li = []
    h = {'User-Agent': '...',
         'Referer': '...',
         'cookie': '...'}
    for d in pd.date_range(startdate, enddate):
        str_d = d.strftime("%Y.%m.%d")
        page = 1
        print(str_d)
        while True:
            start = (page - 1) * 10 + 1
            print(page)
            URL = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query={0}&sort=2&photo=0&field=0&pd=3&ds={1}&de={2}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:from{3}to{4},a:all&start={5}".format(keyword, str_d, str_d, str_d.replace(".", ""), str_d.replace(".", ""), start)
            res = requests.get(URL, headers=h)
            soup = BeautifulSoup(res.text, "html.parser")
            if soup.select_one(".api_noresult_wrap"):
                break
            news_list = soup.select("ul.list_news li")
            for item in news_list:
                if len(item.select("div.info_group a")) == 2:
                    li.append(news(item.select("div.info_group a")[1]['href']))
            page = page + 1
    return pd.DataFrame(li, columns=['title', 'date', 'media', 'content', 'url'])

# 키워드, 검색 기간 설정
result_df = news_list('테슬라', '2022.05.01', '2022.05.31')
result_df

# 파일 이름 설정
result_df.to_csv('[2022-May]news_data.csv', index=False, encoding='utf-8-sig')

2022.05.01
1
2
3
4
5
6
7
8
9
10
11
12
13
2022.05.02
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
2022.05.03
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
2022.05.04
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
2022.05.05
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
2022.05.06
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
2022.05.07
1
2
3
4
5
6
7
8
9
10
2022.05.08
1
2
3
4
5
6
7
8
9
2022.05.09
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2022.05.10
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
2022.05.11
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2022.05.12
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
2022.05.13
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
2022.05.14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
2022.05.15
1
2
3
4
5
6
7
8
9
2022.05.16
1


In [2]:
#csv파일 저장위치
import os
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /content


In [3]:
import pandas as pd

df = pd.read_csv('[2022-May]news_data.csv')
df

,title,date,media,content,url
0,"'제2의 테슬라'라는 이 기업, 어디서 사나요?",2022-05-01 06:01:05,조선일보,2022년 4월 8일(현지시각) 미국 플로리다주(州) 케네디 우주센터에서 민간인 고...,https://n.news.naver.com/mnews/article/023/000...
1,머스크 “트위터 엄청난 잠재력…그 어느 때보다 낫게 만들 것”,2022-05-01 06:02:03,한경비즈니스,[이 주의 한마디]\n\n\n\n엘론 머스크. 사진=연합뉴스전기차 업체 테슬라의 엘...,https://n.news.naver.com/mnews/article/050/000...
2,美 반도체 '3배 레버리지' ETF에 몰린 4월 서학개미,2022-05-01 06:15:00,뉴스1,SOXL 8107억원 순매수하면서 상위 1위에 올라2위 종목도 3배 레버리지…나스닥...,https://n.news.naver.com/mnews/article/421/000...
3,2년 '업적'의 부메랑…中시진핑 올해 최대 위기 맞나? [차이나는 중국],2022-05-01 06:27:00,머니투데이,[편집자주] 차이 나는 중국을 불편부당한 시선으로 바라봅니다.\n\n\n\n시진핑 ...,https://n.news.naver.com/mnews/article/008/000...
4,팔각형·나비형으로 독특해지는 車 핸들… “운전에 위험” 지적도,2022-05-01 07:01:01,조선비즈,자동차 업계가 독특한 스티어링 휠(Steering Wheel·운전대)을 속속 선보이...,https://n.news.naver.com/mnews/article/366/000...
...,...,...,...,...,...
3034,"머스크 ""중국은 전기차 분야 세계 선두""…테슬라 세계 판매량 절반이 中 차지",2022-05-31 19:32:13,아시아경제,"테슬라 작년 중국 판매량, 세계 판매량의 절반 이상\n\n\n\n일론 머스크 테슬라...",https://n.news.naver.com/mnews/article/277/000...
3035,내일부터 상하이 봉쇄 해제…외국 자금·인력 ‘탈중국’,2022-05-31 19:45:56,KBS,[앵커] 중국의 경제 수도 상하이가 내일 코로나19 봉쇄를 풀고 사회 경제 활동 정...,https://n.news.naver.com/mnews/article/056/001...
3036,중국 상하이 봉쇄 풀리지만…외국 자금·인력 ‘탈중국’,2022-05-31 21:48:43,KBS,[앵커]중국 상하이가 내일(1일) 코로나 봉쇄를 풀고 정상화에 시동을 겁니다. 하지...,https://n.news.naver.com/mnews/article/056/001...
3037,테슬라 2분기 차량인도대수 월가 예상치보다 낮을듯,2022-05-31 22:54:01,한국경제,"""70%~75% 가동율 고려시 2분기 최대 26만대 예상""월가 애널들 예상치는 28...",https://n.news.naver.com/mnews/article/015/000...


In [4]:
df.drop(['media', 'url'], axis=1, inplace = True)
df

,title,date,content
0,"'제2의 테슬라'라는 이 기업, 어디서 사나요?",2022-05-01 06:01:05,2022년 4월 8일(현지시각) 미국 플로리다주(州) 케네디 우주센터에서 민간인 고...
1,머스크 “트위터 엄청난 잠재력…그 어느 때보다 낫게 만들 것”,2022-05-01 06:02:03,[이 주의 한마디]\n\n\n\n엘론 머스크. 사진=연합뉴스전기차 업체 테슬라의 엘...
2,美 반도체 '3배 레버리지' ETF에 몰린 4월 서학개미,2022-05-01 06:15:00,SOXL 8107억원 순매수하면서 상위 1위에 올라2위 종목도 3배 레버리지…나스닥...
3,2년 '업적'의 부메랑…中시진핑 올해 최대 위기 맞나? [차이나는 중국],2022-05-01 06:27:00,[편집자주] 차이 나는 중국을 불편부당한 시선으로 바라봅니다.\n\n\n\n시진핑 ...
4,팔각형·나비형으로 독특해지는 車 핸들… “운전에 위험” 지적도,2022-05-01 07:01:01,자동차 업계가 독특한 스티어링 휠(Steering Wheel·운전대)을 속속 선보이...
...,...,...,...
3034,"머스크 ""중국은 전기차 분야 세계 선두""…테슬라 세계 판매량 절반이 中 차지",2022-05-31 19:32:13,"테슬라 작년 중국 판매량, 세계 판매량의 절반 이상\n\n\n\n일론 머스크 테슬라..."
3035,내일부터 상하이 봉쇄 해제…외국 자금·인력 ‘탈중국’,2022-05-31 19:45:56,[앵커] 중국의 경제 수도 상하이가 내일 코로나19 봉쇄를 풀고 사회 경제 활동 정...
3036,중국 상하이 봉쇄 풀리지만…외국 자금·인력 ‘탈중국’,2022-05-31 21:48:43,[앵커]중국 상하이가 내일(1일) 코로나 봉쇄를 풀고 정상화에 시동을 겁니다. 하지...
3037,테슬라 2분기 차량인도대수 월가 예상치보다 낮을듯,2022-05-31 22:54:01,"""70%~75% 가동율 고려시 2분기 최대 26만대 예상""월가 애널들 예상치는 28..."


In [5]:
df['content'][0]

'2022년 4월 8일(현지시각) 미국 플로리다주(州) 케네디 우주센터에서 민간인 고객 3명과 전직 우주비행사를 태운 우주선 크루 드래곤(Crew Dragon)이 굉음을 내며 하늘을 향해 날아 올랐습니다. 우주선의 목적지는 국제우주정거장(ISS·International Space Station). 정부가 아닌 민간 기업이 주도한 첫 국제우주정거장 방문 프로젝트였습니다.여행비로 각각 5500만달러(약 675억원)를 낸 고객들은 미국 부동산 투자 사업가 래리 코너와 캐나다 금융가 마크 패시, 이스라엘 공군 조종사 출신 기업인 에이탄 스티브였습니다.\xa0 이들은 8일간 우주정거장에 머물다 4월 18일 지구로 무사히 돌아왔습니다. 어릴 적 공상과학소설에서나 봤던 민간인 우주여행 시대가 열린 것입니다.\n\n\n\n600억원이 넘는 돈을 내고 우주여행을 떠나는 고객들. /SpaceX 유튜브 캡처 이번 우주여행은 상업용 우주정거장 사업을 추진하고 있는 미국 스타트업 액시엄 스페이스가 우주탐사기업 스페이스X(SpaceX)에 의뢰해 성사됐습니다. 미 항공우주국(NASA)이 우주 탐사 프로젝트를 승인했다고 하죠. 스페이스X는 세계 최대 전기차 제조업체 테슬라(Tesla)를 만든 일론 머스크가 2002년 5월 설립한 기업입니다. 테슬라 법인을 세운 게 2003년 7월이니, 스페이스X가 테슬라보다 먼저 태어난 기업인 셈입니다.화성 이주 위해 탄생한 기업스페이스X의 탄생 배경은 흥미롭기로 유명합니다. 머스크는 온라인 결제업체 페이팔을 이베이에 매각해 2002년 1억7000만달러, 우리 돈으로 약 2000억원을 손에 쥐었습니다. 당시 그의 나이는 불과 31살이었습니다.머스크가 인수 대금으로 세운 회사가 바로 스페이스X입니다. 머스크는 인류를 화성으로 이주시킨다는 야망을 갖고 있었습니다. 환경오염 등으로 지구가 거주 불가능한 상태가 되면 화성에 이주해 인류 역사를 이어간다는 게 머스크의 꿈이었습니다. 화성으로 이주하려면 로켓이 필요했죠. 그는 원래 중고 러시아 로켓을 사려 했습니다.

In [6]:
import re, unicodedata
from string import whitespace
pattern_whitespace = re.compile(f'[{whitespace}]+')

df['content'] = df['content'].fillna('').astype(str)

# TBD: newline도 space로 대체하면 괜찮은가?
df['content'] = df['content'].str.replace(
    pattern_whitespace, ' '
).map(lambda x: unicodedata.normalize('NFC', x)).str.strip()

In [7]:
df['content'][0]

'2022년 4월 8일(현지시각) 미국 플로리다주(州) 케네디 우주센터에서 민간인 고객 3명과 전직 우주비행사를 태운 우주선 크루 드래곤(Crew Dragon)이 굉음을 내며 하늘을 향해 날아 올랐습니다. 우주선의 목적지는 국제우주정거장(ISS·International Space Station). 정부가 아닌 민간 기업이 주도한 첫 국제우주정거장 방문 프로젝트였습니다.여행비로 각각 5500만달러(약 675억원)를 낸 고객들은 미국 부동산 투자 사업가 래리 코너와 캐나다 금융가 마크 패시, 이스라엘 공군 조종사 출신 기업인 에이탄 스티브였습니다.\xa0 이들은 8일간 우주정거장에 머물다 4월 18일 지구로 무사히 돌아왔습니다. 어릴 적 공상과학소설에서나 봤던 민간인 우주여행 시대가 열린 것입니다. 600억원이 넘는 돈을 내고 우주여행을 떠나는 고객들. /SpaceX 유튜브 캡처 이번 우주여행은 상업용 우주정거장 사업을 추진하고 있는 미국 스타트업 액시엄 스페이스가 우주탐사기업 스페이스X(SpaceX)에 의뢰해 성사됐습니다. 미 항공우주국(NASA)이 우주 탐사 프로젝트를 승인했다고 하죠. 스페이스X는 세계 최대 전기차 제조업체 테슬라(Tesla)를 만든 일론 머스크가 2002년 5월 설립한 기업입니다. 테슬라 법인을 세운 게 2003년 7월이니, 스페이스X가 테슬라보다 먼저 태어난 기업인 셈입니다.화성 이주 위해 탄생한 기업스페이스X의 탄생 배경은 흥미롭기로 유명합니다. 머스크는 온라인 결제업체 페이팔을 이베이에 매각해 2002년 1억7000만달러, 우리 돈으로 약 2000억원을 손에 쥐었습니다. 당시 그의 나이는 불과 31살이었습니다.머스크가 인수 대금으로 세운 회사가 바로 스페이스X입니다. 머스크는 인류를 화성으로 이주시킨다는 야망을 갖고 있었습니다. 환경오염 등으로 지구가 거주 불가능한 상태가 되면 화성에 이주해 인류 역사를 이어간다는 게 머스크의 꿈이었습니다. 화성으로 이주하려면 로켓이 필요했죠. 그는 원래 중고 러시아 로켓을 사려 했습니다. 러시아로 날

In [8]:
# 바이라인 제거
'''
...했다. 지역=ㅇㅇㅇ 기자, 이메일 주소, ⓒ©, www.example.com
'''

'\n...했다. 지역=ㅇㅇㅇ 기자, 이메일 주소, ⓒ©, www.example.com\n'

In [9]:
# 1. 이메일과 url을 먼저 제거
# 2. 마지막 문장 중 바이라인으로 추정되는 것을 찾아 제거
# TODO 바이라인 제거가 깔끔하게 되지 않음. 예외 찾아 추가 작업 필요
# r'\.([^\.]*(?:(?#email)(?:[-_0-9a-z]+(?:\.[\+0-9a-z]+)*@[0-9a-z]+\.[-_0-9a-z]+(?:\.[0-9a-z]+)*)|(?#url)(?:(?:https?:\/\/)?[0-9a-z]+\.[-_0-9a-z]+(?:\.[0-9a-z]+)*)|기자|특파원|교수|작가|대표|논설|팀장|장관|원장|연구원|이사장|위원|실장|차장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|@|\/|=|▶|무단|전재|재배포|금지)[^\.]*)$'

def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('.', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()

    return result

df['content'] = df['content'].map(clean_byline)

In [10]:
df['content'][0]

'2022년 4월 8일(현지시각) 미국 플로리다주(州) 케네디 우주센터에서 민간인 고객 3명과 전직 우주비행사를 태운 우주선 크루 드래곤(Crew Dragon)이 굉음을 내며 하늘을 향해 날아 올랐습니다. 우주선의 목적지는 국제우주정거장(ISS·International Space Station). 정부가 아닌 민간 기업이 주도한 첫 국제우주정거장 방문 프로젝트였습니다.여행비로 각각 5500만달러(약 675억원)를 낸 고객들은 미국 부동산 투자 사업가 래리 코너와 캐나다 금융가 마크 패시, 이스라엘 공군 조종사 출신 기업인 에이탄 스티브였습니다.\xa0 이들은 8일간 우주정거장에 머물다 4월 18일 지구로 무사히 돌아왔습니다. 어릴 적 공상과학소설에서나 봤던 민간인 우주여행 시대가 열린 것입니다. 600억원이 넘는 돈을 내고 우주여행을 떠나는 고객들. /SpaceX 유튜브 캡처 이번 우주여행은 상업용 우주정거장 사업을 추진하고 있는 미국 스타트업 액시엄 스페이스가 우주탐사기업 스페이스X(SpaceX)에 의뢰해 성사됐습니다. 미 항공우주국(NASA)이 우주 탐사 프로젝트를 승인했다고 하죠. 스페이스X는 세계 최대 전기차 제조업체 테슬라(Tesla)를 만든 일론 머스크가 2002년 5월 설립한 기업입니다. 테슬라 법인을 세운 게 2003년 7월이니, 스페이스X가 테슬라보다 먼저 태어난 기업인 셈입니다.화성 이주 위해 탄생한 기업스페이스X의 탄생 배경은 흥미롭기로 유명합니다. 머스크는 온라인 결제업체 페이팔을 이베이에 매각해 2002년 1억7000만달러, 우리 돈으로 약 2000억원을 손에 쥐었습니다. 당시 그의 나이는 불과 31살이었습니다.머스크가 인수 대금으로 세운 회사가 바로 스페이스X입니다. 머스크는 인류를 화성으로 이주시킨다는 야망을 갖고 있었습니다. 환경오염 등으로 지구가 거주 불가능한 상태가 되면 화성에 이주해 인류 역사를 이어간다는 게 머스크의 꿈이었습니다. 화성으로 이주하려면 로켓이 필요했죠. 그는 원래 중고 러시아 로켓을 사려 했습니다. 러시아로 날

In [11]:
# 전처리한 데이터 csv 파일로 저장
df['content'].to_csv('[2022-May]news_data_preprocessing.csv', index=False, encoding='utf-8-sig')

In [12]:
# gensim 을 이용하여 토큰화, LDA모델링

None